In [81]:
# imports
import sys

PATH = os.path.join(os.getcwd(), '..', '..', '.local', 'share', 'virtualenvs', 'cityspire-ds-h-NIlzhGdy', 'lib', 'python3.8', 'site-packages')
sys.path.append(PATH)

import os
import pandas as pd
import sqlalchemy

In [92]:
# reading in the data
crime_data_dict = {}
rows_to_skip_dict = {'2010': 7,
                    '2011': 7,
                    '2012': 7,
                    '2013': 10,
                    '2014': 9,
                    '2015': 10,
                    '2016': 9,
                    '2017': 10,
                    '2018': 10,
                    '2019': 8}
for year in range(2010, 2020):
    year = str(year)
    FILE_PATH = os.path.join(os.getcwd(), '..', 'raw_data', f'city_crime_data_{year}.csv')
    crime_data_dict[year] = pd.read_csv(FILE_PATH, skiprows=3, skipfooter=rows_to_skip_dict[year])
    crime_data_dict[year].columns = ['state_name', 'city_name', 'violent_crime', 'murder_and_nonnegligent_homicide',
                'rape', 'robbery', 'aggravated_assault', 'property_crime', 'burglary',
                'larceny_theft', 'motor_vehicle_theft', 'arson']

In [93]:
crime_data_dict['2019']

,state_name,city_name,violent_crime,murder_and_nonnegligent_homicide,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motor_vehicle_theft,arson
0,ALABAMA3,Hoover,114,4.0,15,27,68,"1,922",128,"1,694",100,2
1,ALASKA,Anchorage,"3,581",32.0,540,621,"2,388","12,261","1,692","9,038","1,531",93
2,NaN,Bethel,130,1.0,47,3,79,132,20,84,28,12
3,NaN,Bristol Bay Borough,2,0.0,0,0,2,20,5,8,7,0
4,NaN,Cordova,0,0.0,0,0,0,7,1,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8100,NaN,Sheridan,9,0.0,4,0,5,369,75,278,16,3
8101,NaN,Thermopolis5,13,0.0,0,0,13,34,7,22,5,0
8102,NaN,Torrington,13,0.0,4,1,8,48,8,40,0,0
8103,NaN,Wheatland,7,0.0,1,0,6,72,24,45,3,0


In [107]:
# data cleaning
for key, df in crime_data_dict.items():
    df['state_name'] = df['state_name'].fillna(method='ffill')
    df['state_name'] = df['state_name'].apply(lambda x: ''.join([letter for letter in x if not letter.isdigit()]))
    df['state_name'] = df['state_name'].apply(lambda x: x.capitalize())
    df['city_name'] = df['city_name'].apply(lambda x: ''.join([letter for letter in str(x) if not letter.isdigit()]))
    crime_data_dict[f'{key}'] = crime_data_dict[f'{key}'].fillna('n/a')
    for column in df.columns[2:]:
        crime_data_dict[f'{key}'][f'{column}'] = crime_data_dict[f'{key}'][f'{column}'].apply(lambda x: str(x))
        crime_data_dict[f'{key}'][f'{column}'] = crime_data_dict[f'{key}'][f'{column}'].apply(
            lambda x: float(x.replace(',', '')) if x != 'n/a' else x)


In [112]:
# merging in city_id


float

In [73]:
DATABASE_URL = os.getenv('PRODUCTION_DATABASE_URL')
query = '''
        SELECT Cities.city_id, Cities.city_name, States.state_name
        FROM CITIES
        LEFT JOIN STATES ON CITIES.state_id=STATES.state_id
        '''
cities = pd.read_sql(query, DATABASE_URL)
cities

,city_id,city_name,state_name
0,0100124,Abbeville ...,Alabama ...
1,0100460,Adamsville ...,Alabama ...
2,0100484,Addison ...,Alabama ...
3,0100676,Akron ...,Alabama ...
4,0100820,Alabaster ...,Alabama ...
...,...,...,...
18248,5681300,Wamsutter ...,Wyoming ...
18249,5683040,Wheatland ...,Wyoming ...
18250,5684925,Worland ...,Wyoming ...
18251,5685015,Wright ...,Wyoming ...


In [74]:
df

,state_name,city_name,violent_crime,murder_and_nonnegligent_homicide,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motor_vehicle_theft,arson,city_id
0,Alabama,Abbeville,18,0,2,0,16,49,14,33,2,n/a,2
1,Alabama,Adamsville,19,0,1,4,14,289,42,230,17,n/a,2
2,Alabama,Alabaster,92,0,2,10,80,579,56,497,26,n/a,2
3,Alabama,Albertville,24,0,6,10,8,802,194,492,116,n/a,2
4,Alabama,Alexander City,314,2,5,15,292,610,92,484,34,n/a,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9247,Wyoming,Sheridan,17,1,2,0,14,313,39,261,13,1,2
9248,Wyoming,Thermopolis,13,0,0,0,13,64,10,48,6,0,2
9249,Wyoming,Torrington,35,0,0,0,35,73,12,57,4,0,2
9250,Wyoming,Wheatland,3,0,0,0,3,56,19,33,4,0,2


In [80]:
for key, df in crime_data_dict.items():
    print(key)

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
